# Lending Club Case Study

### Group Facilitator - Budhaditya Saha
### Group Member - Manohar Simons

## Step 1: Data understanding

Import the required libraries

In [2]:
import chardet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import types

Determine the encoding of the csv file

In [3]:
def detect_encoding(filename):
    with open(filename, mode="rb") as rawdata:
        result = chardet.detect(rawdata.read())

    print(result)

# Uncomment to determine the encoding of the file
#detect_encoding("loan.csv")

Read the csv file using the appropriate encoding

In [4]:
loan = pd.read_csv("loan.csv", encoding="utf-8")

/Users/manoharsimons/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:
# determine the number of rows and columns
loan.shape

In [ ]:
loan.describe()

In [ ]:
# check the datatypes of various columns
loan.dtypes

Truncate and modify the data type of the interest rate column

In [ ]:
loan["int_rate"] = loan["int_rate"].str[:-1].astype(float)
loan["int_rate"].dtype
print(loan["int_rate"].head())

In [ ]:
loan.emp_length.value_counts(dropna=False)

Convert employee length to numeric

Assumption: The employment length column has 2.67% n/a values. Also all but 18 rows that have empolyment length as n/a also have the employer title as blank. Which probably implies that these applicants are not employed and we can treat these values as 0's.

In [ ]:
loan["emp_length"] = loan["emp_length"].fillna("0 years")

loan[loan["emp_length"]=="0 years"].shape

In [ ]:
loan["emp_length"]=loan.emp_length.str.replace('[\+\<]','').str.strip().str.replace('["year","years"]','', regex=True)
loan['emp_length']=(loan['emp_length']).astype(int)
loan.emp_length.dtype

The target column is the loan status. We retain the fully paid and charged off loans and drop loans that have a current status

In [ ]:
loan = loan[loan["loan_status"] != "Current"]

Convert fully paid and charged off loans to 0's an 1's

In [ ]:
loan["loan_status"] = (loan["loan_status"] == "Charged Off").astype(int)

## Step 2: Data cleaning

Determine the number of null columns

In [ ]:
# determine the number of null columns
pd.set_option('display.max_rows', None)
null_columns = loan.count(axis="rows")
null_columns = null_columns[null_columns == 0]
null_columns.shape

There are 54 columns that do not have any data. So let's drop them.

In [ ]:
loan = loan.drop(columns=null_columns.index)
loan.shape

These are the empty columns that we have dropped

In [ ]:
null_columns

Check for columns that have only 0's in it

In [ ]:
loan.columns[(loan == 0).all()]

Drop the columns that have all 0's in it

In [ ]:
loan = loan.drop(columns=loan.columns[(loan == 0).all()])
loan.shape

Defining a function to drop unecessary columns

In [ ]:
def drop_columns_on_dataframe(df, columns):
    initial_count = df.shape[1]
    df = df.drop(columns=columns_to_drop)
    print("Number of columns dropped: {}".format(initial_count - df.shape[1]))
    print("Columns remaining: {}".format(df.shape[1]))
    return df

Dropping a few other columns that we do not need for our analysis

In [ ]:
columns_to_drop = ["emp_title", "pymnt_plan", "url", "desc", "title", "zip_code"]
loan = drop_columns_on_dataframe(loan, columns_to_drop)

Droping the customer behavioural variables

In [ ]:
columns_to_drop = ["delinq_2yrs", "earliest_cr_line", "inq_last_6mths", "mths_since_last_delinq", "mths_since_last_record", "open_acc", "pub_rec", "revol_bal", "revol_util", "total_acc", "initial_list_status", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "last_credit_pull_d", "collections_12_mths_ex_med", "chargeoff_within_12_mths", "pub_rec_bankruptcies", "tax_liens"]
loan = drop_columns_on_dataframe(loan, columns_to_drop)

Check all the other columns for missing values

In [ ]:
nullseries = loan.isnull().sum()
print(nullseries[nullseries > 0])
print(loan.shape)

## Step 3: Data Analysis 

### Univariate analysis on categorical variable

Define a function to display barplots

In [ ]:
def showBarplot(x, y, data, estimator, title, xlabel, ylabel, xticks, figsize):
    plt.figure(figsize=figsize)
    if type(estimator) == type(lambda x:x):
        ax = sns.barplot(x=x, y=y, data=data, estimator=estimator)
    else:
        ax = sns.barplot(x=x, y=y, data=data)
    max = 0
    for p in ax.patches:
        ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2.0, p.get_height()),
         ha='center', va='center', fontsize=11, color='gray', xytext=(0, 20),
         textcoords='offset points')
        if p.get_height() > max:
            max = p.get_height()
    ax.set_ylim(0, 1.15*max )
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    if xticks != []:
        ax.set_xticklabels(xticks)
    plt.show(ax)

Define a function to display boxplots to compare good and bad loans

In [ ]:
#Create a user defined function to show a box plot that comapres bad loans with good loans
def boxplot(xparameter, yparameter, showfliers):
    sns.boxplot(y=yparameter, 
                x=xparameter,
                data=loan, 
                hue="loan_status",
                showfliers=showfliers)
    plt.show()

Define a function to display barplots

In [ ]:
#create user defined function for bar plots for unicariate analysis
def snsplot(column_name, size):
    plt.figure(figsize=size)
    sns.barplot(x=column_name, y="loan_status", data=loan)
    plt.show()

In [ ]:
showBarplot("loan_status", 
            "loan_status", 
            loan, 
            lambda x: len(x)*100/float(len(loan)), 
            "Percentage of loans", 
            "Loan Status",
            "Percentage of non-defaulters and defaulters",
            ["Fully Paid", "Charged Off"],
            [10,5])

#### Term of the loan

In [ ]:
showBarplot("term", 
            "loan_status", 
            loan, 
            None, 
            "Percentage of Loan Defaults vs Term",
            "Term",
            "Percentage of loans",
            [],
            [10,5])

*36 months:* 11% of applicants default

60 months: 25% of applicants default 

#### Grade of the loan

In [ ]:
showBarplot("grade", 
            "loan_status", 
            loan.sort_values("grade"), 
            None, 
            "Percentage of defaulters vs Loan Grade", 
            "Percentage of defaulters",
            "Loan Grade",
            [],
            [10,5])

The percentage of defaults increases as the loan grade moves from A to G 

#### Purpose of the loan

In [ ]:
loan['purpose'].value_counts()

In [ ]:
# top 5 loan purpose

top5loan = loan[(loan.purpose=='debt_consolidation') |
            (loan.purpose=='credit_card') |
            (loan.purpose=='other') |
            (loan.purpose=='home_improvement') |
            (loan.purpose=='major_purchase')]

In [ ]:
top5loan['purpose'].value_counts()

In [ ]:
showBarplot("purpose", 
            "loan_status", 
            loan.sort_values("purpose"), 
            None, 
            "Top 5 purpose of taking a loan for defaulters", 
            "Percentage of applicants",
            "Loan Purpose",
            [],
            [20,10])

Loans taken for the purpose of a small business have the highes rate of default

### Univariate analysis on numerical variables

#### Principal of the loan

In [ ]:
loan.loan_amnt.describe()

In [ ]:
loan['loan_category'] = pd.qcut(loan['loan_amnt'], q=10, precision=0)

In [ ]:
snsplot('loan_category', [20,5])

#### Debt to income ratio

In [ ]:
loan['dti'].describe()

In [ ]:
loan['dti_category'] = pd.qcut(loan['dti'], q=4, precision=0)

In [ ]:
snsplot('dti_category', [20,10])

In [ ]:
boxplot("term", "dti", True)

#### Annual Income

In [ ]:
loan['annual_inc_cat'] = pd.qcut(loan['annual_inc'], q=4, precision=0)

In [ ]:
loan.head()

In [ ]:
snsplot('annual_inc_cat', [20,10])

People with a lower annual income tend to default

In [ ]:
loan['emp_length'].value_counts()

In [ ]:
snsplot('emp_length',[20,15])

People who have 0 years of employment have a strong tendancy to default

### Bivariate analysis

#### Grade

In [ ]:
plt.figure(figsize=[20,10])
sns.barplot(x="term", y="loan_status", hue="grade", data=loan)
plt.show()

#### Subgrade

In [ ]:
plt.figure(figsize=[20,10])
sns.barplot(x="grade", y="loan_status", hue="sub_grade", data=loan)
plt.show()

#### Verification Status

In [ ]:
plt.figure(figsize=[20,10])
sns.barplot(x="term", y="loan_status", hue="verification_status", data=loan)
plt.show()

Verification status does not seem to be a good predictor of a default

#### Interest rates

In [ ]:
loan['int_rate_category'] = pd.qcut(loan['int_rate'], q=4, precision=0)

In [ ]:
snsplot('int_rate_category', [20,5])

In [ ]:
boxplot("term", "int_rate", True)

Loans that are defaulted have a higher rate of interest than loans that are repayed

#### Principal

In [ ]:
boxplot("term", "loan_amnt", True)

The principal does not seem to be a predictor of a default

#### Installment

In [ ]:
boxplot("term", "installment", True)

People paying higher installment on longer term loans have a larger tendancy to default

#### Annual Income

In [ ]:
boxplot("term", "annual_inc", False)

#### Home Ownership

In [ ]:
boxplot("home_ownership", "annual_inc", False)

People who take shorter term loans tend to have a higher annual incomes than those who take longer term loans across all home ownership categories

#### Purpose of Loan v/s Interest rate v/s employment length

In [ ]:
plt.figure(figsize=[20,6])
sns.barplot(x="purpose", y="emp_length", hue="int_rate_category", data=loan)
plt.show()

In [ ]:
plt.figure(figsize=[20,20])
sns.barplot(x="purpose", y="emp_length", hue="grade", data=loan)
plt.show()

## 4.1 Most Important Predictors: Categorical Variables

This function returns the standard deviation of the default rate of a value in a categorical column

In [ ]:
def getStandardDeviationOfDefaultRateCategorical(column):
    unique_values = loan[column].unique()
    default_rate = []
    for value in unique_values:
        default_rate.append(len(loan[(loan[column]==value) & (loan["loan_status"]==1)])/len(loan[loan[column]==value]))

    df = pd.DataFrame(default_rate,columns=["default_rate"])
    return round(df["default_rate"].std(), 2)

This function return the standard deviation of a list of categorical columns sorted by the standard deviation in descending order

In [ ]:
def standardDeviationOfCategoricalColumns(column_list):
    deviation = {}
    for column in column_list:
        deviation[column] = getStandardDeviationOfDefaultRateCategorical(column)
    return {k: v for k, v in sorted(deviation.items(), key=lambda item: item[1], reverse=True)}


In [ ]:
standardDeviationOfCategoricalColumns(["term", "grade", "purpose", "sub_grade", "emp_length", "home_ownership", "verification_status", "addr_state"])

The significant categorical predictors are grade, sub grade and term

## 4.2 Most Important Predictors: Numerical variables

This function returns the standard deviation of the default rate of a value in a numeric column

In [ ]:
def getStandardDeviationOfDefaultRateNumeric(column):
    loan[column + "_cut"] = pd.qcut(loan[column], q=4, precision=0)
    unique_values = loan[column + "_cut"].unique()
    default_rate = []
    for value in unique_values:
        default_rate.append(len(loan[(loan[column + "_cut"]==value) & (loan["loan_status"]==1)])/len(loan[loan[column + "_cut"]==value]))

    df = pd.DataFrame(default_rate,columns=["default_rate"])
    return round(df["default_rate"].std(), 2)

This function return the standard deviation of a list of numeric columns sorted by the standard deviation in descending order

In [ ]:
def standardDeviationOfNumericColumns(column_list):
    deviation = {}
    for column in column_list:
        deviation[column] = getStandardDeviationOfDefaultRateNumeric(column)
    return {k: v for k, v in sorted(deviation.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
standardDeviationOfNumericColumns(["loan_amnt", "funded_amnt", "funded_amnt_inv", "int_rate", "installment", "annual_inc"])

The significant numerical predictor is interest rate

# End of case study